In [1]:
from ch2.data import *
from ch2.squeal import *
from ch2.lib.date import to_time
s = session('-v 5')
a = df(s.query(ActivityJournal).join(ActivityGroup).filter(ActivityGroup.name == 'Bike'))
x = a.loc[(a.start >= to_time('2017-07-01')) & (a.start < to_time('2017-07-02'))]
print(x)
ride = waypoints(s, x.id.iloc[0,0], 'Spherical Mercator X', 'Spherical Mercator Y', 'Latitude', 'Longitude', 'Distance')
print(ride)

    INFO: Using database at /home/andrew/.ch2/database.sqlr


         id      id  type  activity_group_id            name  \
168  175381  175381     2                  1  2017-07-01-fun   

                                             fit_file  \
168  /home/andrew/archive/fit/bike/2017-07-01-fun.fit   

                        start                    finish  
168 2017-07-01 12:05:05+00:00 2017-07-01 13:24:37+00:00  


NameError: name 'waypoints' is not defined

In [3]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models.widgets import Slider, PreText

output_notebook()
width, height = 800, 800

def modify_doc(doc):

    t1 = PreText(height=20, width=200)
    t2 = PreText(height=20, width=200)
    t3 = PreText(height=20, width=100)
    s1 = Slider(start=0, end=len(ride), value=0, title='Start')
    s2 = Slider(start=1, end=len(ride)-1, value=len(ride), title='Length')
    f = figure(plot_width=width, plot_height=height, x_axis_type='mercator', y_axis_type='mercator')
    c = column(row(s1, s2), row(t1, t2, t3), f)

    def mkplot(l1, l2):
        l2 = min(len(ride)-1, l1+l2)
        t1.text = '%9.5f,%9.5f' % (ride.iloc[l1]['Longitude'], ride.iloc[l1]['Latitude'])
        t2.text = '%9.5f,%9.5f' % (ride.iloc[l2]['Longitude'], ride.iloc[l2]['Latitude'])
        t3.text = '%4.1fkm' % ((ride.iloc[l2]['Distance'] - ride.iloc[l1]['Distance']) / 1000)
        f = figure(plot_width=width, plot_height=height, x_axis_type='mercator', y_axis_type='mercator')
        f.circle(x='Spherical Mercator X', y='Spherical Mercator Y', source=ride[l1:l2])
        f.add_tile(CARTODBPOSITRON)
        c.children[2] = f

    s1.on_change('value', lambda attr, old, new: mkplot(s1.value, s2.value))
    s2.on_change('value', lambda attr, old, new: mkplot(s1.value, s2.value))
    doc.add_root(c)

show(modify_doc)


Loading BokehJS ...

In [4]:
from ch2.squeal.database import connect
from ch2.squeal.tables.segment import Segment
from ch2.squeal.tables.activity import ActivityGroup

g = s.query(ActivityGroup).filter(ActivityGroup.name == 'Bike').one()
print(g)
x = Segment(start=(-70.61813,-33.41536), finish=(-70.63340,-33.42655), distance=4400,
            activity_group=g,
            name='San Cristobal', description='Climb up San Cristobal in Parque Metropolitana')
#s.add(x)

INFO:sqlalchemy.engine.base.Engine:SELECT activity_group.id AS activity_group_id, activity_group.name AS activity_group_name, activity_group.description AS activity_group_description, activity_group.sort AS activity_group_sort 
FROM activity_group 
WHERE activity_group.name = ?
INFO:sqlalchemy.engine.base.Engine:('Bike',)


ActivityGroup "Bike"
